## Import libraries

In [1]:
import os
from twilio.rest import Client
from twilio_config import PHONE_NUMBER, TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, API_KEY_WAPI
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

## URL configuration parameters

In [2]:
city = 'madrid'
api_key = API_KEY_WAPI
days = '2'
air_quality = 'yes'
weather_alert = 'no'
url_weather = 'http://api.weatherapi.com/v1/forecast.json?key=' + api_key + '&q=' + city + '&days=' + days + '&aqi=' + air_quality + '&alerts=' + weather_alert

## Weather request

In [3]:
response = requests.get(url_weather).json()

In [4]:
response

{'location': {'name': 'Madrid',
  'region': 'Madrid',
  'country': 'Spain',
  'lat': 40.4,
  'lon': -3.68,
  'tz_id': 'Europe/Madrid',
  'localtime_epoch': 1721663364,
  'localtime': '2024-07-22 17:49'},
 'current': {'last_updated_epoch': 1721663100,
  'last_updated': '2024-07-22 17:45',
  'temp_c': 35.0,
  'temp_f': 95.0,
  'is_day': 1,
  'condition': {'text': 'Sunny',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
   'code': 1000},
  'wind_mph': 2.2,
  'wind_kph': 3.6,
  'wind_degree': 103,
  'wind_dir': 'ESE',
  'pressure_mb': 1015.0,
  'pressure_in': 29.97,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 8,
  'cloud': 0,
  'feelslike_c': 33.2,
  'feelslike_f': 91.7,
  'windchill_c': 35.0,
  'windchill_f': 95.0,
  'heatindex_c': 33.2,
  'heatindex_f': 91.7,
  'dewpoint_c': -4.5,
  'dewpoint_f': 23.9,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 9.0,
  'gust_mph': 6.7,
  'gust_kph': 10.8,
  'air_quality': {}},
 'forecast': {'forecastday': [{'date': '2024-07-22',


In [5]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [6]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [7]:
#Date
response['forecast']['forecastday'][0]['date']

'2024-07-22'

In [8]:
#Hour
int (response['forecast']['forecastday'][0]['hour'][1]['time'].split(' ')[1].split(':')[0])

1

In [9]:
#Condition
response['forecast']['forecastday'][0]['hour'][1]['condition']['text']

'Clear '

In [10]:
#Temperature
response['forecast']['forecastday'][0]['hour'][1]['temp_c']

20.6

In [11]:
#Rain
response['forecast']['forecastday'][0]['hour'][1]['will_it_rain']

0

In [12]:
#Rain_probability
response['forecast']['forecastday'][0]['hour'][1]['chance_of_rain']

0

## Dataframe

In [13]:
#Auxiliar function to get weather variables by hour
def get_forecast(response, i):
    date = response['forecast']['forecastday'][0]['date']
    hour = int (response['forecast']['forecastday'][0]['hour'][i]['time'].split(' ')[1].split(':')[0])
    condition = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    temperature = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    rain_probability = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    return date, hour, condition, temperature, rain, rain_probability 


In [14]:
data = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour = 'red'):
    data.append(get_forecast(response, i))

100%|██████████| 24/24 [00:00<?, ?it/s]


In [15]:
#Dataframe columns
col = ['date', 'hour', 'condition', 'temp', 'rain', 'rain_prob']

In [16]:
#Dataframe creation
df = pd.DataFrame(data, columns=col)

In [17]:
df.head()

,date,hour,condition,temp,rain,rain_prob
0,2024-07-22,0,Clear,21.3,0,0
1,2024-07-22,1,Clear,20.6,0,0
2,2024-07-22,2,Clear,19.9,0,0
3,2024-07-22,3,Clear,19.3,0,0
4,2024-07-22,4,Clear,19.0,0,0


In [18]:
#Dataframe filters
temp_max = 32
temp_min = 5
rain = 1
rain_prob = 60

In [19]:
df_filter = df[(df['temp'] >= temp_max) | (df['temp'] <= temp_min) | (df['rain'] == rain) | (df['rain_prob'] >= rain_prob)]

In [20]:
df_filter.dtypes

date          object
hour           int64
condition     object
temp         float64
rain           int64
rain_prob      int64
dtype: object

## SMS send

In [21]:
text = '\nAlert for temperatures above ' + str (temp_max) + ' in ' + city + ': \n\n ' + str (df_filter[['hour', 'temp']].set_index('hour'))

In [22]:
client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

message = client.messages.create(  
  body = text,
  from_ = PHONE_NUMBER,
  to='+34658936280'
)

print('Mensaje enviado ' + message.sid)

Mensaje enviado SMdf407b2c02989907bfd5c702893d4de6
